In [1]:
import numpy as np
import math
import time
from copy import deepcopy
        
        
class Conv3x3:
        
    def __init__(self, num_filters):
        self.num_filters = num_filters
        self.filters = np.random.randn(num_filters, 3, 3) / 9
        #self.bias = np.random.randn(num_filters)/num_filters
        #self.filters_1 = self.filters
        self.t_for_np_loops = 0
        self.t_for_conv = 0
        self.t_back_prop = 0
        #self.stc_indices = np.zeros(self.filters.shape)
        
        
        
    def iterate_regions(self, image):
        
        h, w = image.shape

        for i in range(h - 2):
              for j in range(w - 2):
                im_region = image[i:(i + 3), j:(j + 3)]
                yield im_region, i, j
                
    
    def stc(self):
        rho = 0.5
        
        w = self.filters.shape[1]*self.filters.shape[2]
        
        n = math.floor(rho*w)
        
       # print(n)
        
        d = {0:[],1:[],2:[],3:[],4:[],5:[],6:[],7:[]}
 
    #        self.filters_1 = deepcopy(self.filters)
        
        
        
        for i in range(len(self.filters)):
            
            for _ in range(n): 
        
                min = math.inf
                for j in range(len(self.filters[i])):
                    for k in range(len(self.filters[i][j])):
                        if abs(self.filters[i,j,k]) < min:
                            min = abs(self.filters[i,j,k])
                            min_j = j
                            min_k = k
        
                self.filters[i][min_j][min_k] = math.inf
        
            for j in range(len(self.filters[i])):
                for k in range(len(self.filters[i][j])):
                    if self.filters[i,j,k] == math.inf:
                        self.filters[i,j,k] = 0

        
                    
        for i in range(len(self.filters)):
            
            for j in range(len(self.filters[i])):
            
                for k in range(len(self.filters[i][j])):
            
                    if self.filters[i,j,k] > 0 :
                        self.filters[i,j,k] = 1
#                         #self.stc_indices[i,j,k] = 1
                        d[i].append([j,k,1])
            
                    elif self.filters[i,j,k] < 0 :
                        self.filters[i,j,k] = -1
#                         #self.stc_indices[i,j,k] = 1
                        d[i].append([j,k,-1])
                        
                        
                        
        return d
        

    def forward(self, input):
        
        v = time.time()
       
        d = self.stc()
        
        u = time.time()
        
        self.t_for_np_loops += u-v
        

        
        self.last_input = input

        h, w = input.shape
        
        
        output = np.zeros((h - 2, w - 2, self.num_filters))
        
        x = time.time()

        for i in range(h - 2):
              for j in range(w - 2):
                    
                    im_region = input[i:(i + 3), j:(j + 3)]
                    
                    for k in range(self.num_filters):
                            for ele in d[k]:
                                
                                if ele[2] == 1:
                                    
                                    output[i, j,k] += im_region[ele[0],ele[1]]
                                
                                else:
                                    
                                    output[i, j,k] += -(im_region[ele[0],ele[1]])
                                
                            output[i, j,k] = np.maximum(0,output[i, j,k])
        
        
#         for im_region, i, j in self.iterate_regions(input):
#               output[i, j] = np.sum(im_region * self.filters_1, axis=(1, 2))

        
                
        z = time.time()
        
        self.t_for_conv += z-u
        
        
        
        return output
    

    def backprop(self, d_L_d_out, learn_rate):
        
        #print(d_L_d_out)
        
        bck1 = time.time()
        
        
        d_L_d_filters = np.zeros(self.filters.shape)
        
        

        for im_region, i, j in self.iterate_regions(self.last_input):
              for f in range(self.num_filters):
                d_L_d_filters[f] += d_L_d_out[i, j, f] * im_region

#         d_L_d_b = np.zeros(self.bias.shape)
        
#         for f in range(self.num_filters):
#                 d_L_d_b[f] += np.sum(d_L_d_out[:, :, f])
#         self.bias -= learn_rate * d_L_d_b
        
        self.filters -= learn_rate * d_L_d_filters
        
        self.filters = np.clip(self.filters,-1,1)

        bck2 = time.time()
        
        self.t_back_prop += bck2 - bck1
        
        return None
                    
    
    


In [2]:
import numpy as np

class MaxPool2:
    
    def iterate_regions(self, image):
        h, w, _ = image.shape
        new_h = h // 2
        new_w = w // 2
    
        for i in range(new_h):
            for j in range(new_w):
                im_region = image[(i * 2):(i * 2 + 2), (j * 2):(j * 2 + 2)]
                yield im_region, i, j
    
    def forward(self, input):
    
        self.last_input = input

        h, w, num_filters = input.shape
        output = np.zeros((h // 2, w // 2, num_filters))

        for im_region, i, j in self.iterate_regions(input):
            output[i, j] = np.amax(im_region, axis=(0, 1))

        return output

    def backprop(self, d_L_d_out):
       
        d_L_d_input = np.zeros(self.last_input.shape)

        for im_region, i, j in self.iterate_regions(self.last_input):
            h, w, f = im_region.shape
            amax = np.amax(im_region, axis=(0, 1))

            for i2 in range(h):
                for j2 in range(w):
                      for f2 in range(f):
                        # If this pixel was the max value, copy the gradient to it.
                            if im_region[i2, j2, f2] == amax[f2]:
                                  d_L_d_input[i * 2 + i2, j * 2 + j2, f2] = d_L_d_out[i, j, f2]
                                    
        d_L_d_input[self.last_input<0] = 0

        return d_L_d_input

In [3]:
import numpy as np

class Softmax:

    def __init__(self, input_len, nodes):
        self.weights = np.random.randn(input_len, nodes) / input_len
        self.biases = np.zeros(nodes)

    def forward(self, input):
        
        self.last_input_shape = input.shape

        input = input.flatten()
        self.last_input = input

        input_len, nodes = self.weights.shape

        totals = np.dot(input, self.weights) + self.biases
        self.last_totals = totals

        exp = np.exp(totals)
        return exp / np.sum(exp, axis=0)

    def backprop(self, d_L_d_out, learn_rate):
        
        for i, gradient in enumerate(d_L_d_out):
            if gradient == 0:
                continue

            t_exp = np.exp(self.last_totals)

            S = np.sum(t_exp)

            d_out_d_t = -t_exp[i] * t_exp / (S ** 2)
            d_out_d_t[i] = t_exp[i] * (S - t_exp[i]) / (S ** 2)

            d_t_d_w = self.last_input
            d_t_d_b = 1
            d_t_d_inputs = self.weights

            d_L_d_t = gradient * d_out_d_t

            d_L_d_w = d_t_d_w[np.newaxis].T @ d_L_d_t[np.newaxis]
            d_L_d_b = d_L_d_t * d_t_d_b
            d_L_d_inputs = d_t_d_inputs @ d_L_d_t

            self.weights -= learn_rate * d_L_d_w
            self.biases -= learn_rate * d_L_d_b

            return d_L_d_inputs.reshape(self.last_input_shape)

In [4]:
from mnist import MNIST

mndata = MNIST(r'C:\Users\krish\OneDrive\Documents\krishna\sem6\DL\mnist')

mndata.gz = True

train_images = np.zeros([400,28,28])
test_images = np.zeros([100,28,28])


images_tr,train_labels = mndata.load_training()
images_tr = np.array(images_tr[:400])
train_labels = np.array(train_labels[:400])
    

images_ts1,test_labels = mndata.load_testing()
images_ts = np.array(images_ts1[:100])
test_labels = np.array(test_labels[0:100])


for i in range(400):
    train_images[i] = images_tr[i].reshape(28,28) 
    
for i in range(100):
    test_images[i] = images_ts[i].reshape(28,28)
    
#print(train_images[0])

In [5]:
import mnist
import numpy as np
import time

conv = Conv3x3(8)
pool = MaxPool2()                  
softmax = Softmax(13 * 13 * 8, 10) 

def forward(image, label): 
    
    out = conv.forward((image / 255) )
    out = pool.forward(out)
    out = softmax.forward(out)
    
    
    loss = -np.log(out[label])
    acc = 1 if np.argmax(out) == label else 0

    return out, loss, acc

def train(im, label, lr=.005):
   
    out, loss, acc = forward(im, label)

    gradient = np.zeros(10)
    gradient[label] = -1 / out[label]

    gradient = softmax.backprop(gradient, lr)
    gradient = pool.backprop(gradient)
    gradient = conv.backprop(gradient, lr)
    
    return loss, acc


t1 = time.time()


for epoch in range(3):
    print('--- Epoch %d ---' % (epoch + 1))

    permutation = np.random.permutation(len(train_images))
    train_images = train_images[permutation]
    train_labels = train_labels[permutation]
    
    loss = 0
    num_correct = 0
    for i, (im, label) in enumerate(zip(train_images, train_labels)):
        l, acc = train(im, label)
        loss += l
        num_correct += acc
        
    print('after epoch' + str(epoch+1) + 'loss is ' + str(loss/1000) + 'accuracy is' + str(num_correct/1000))

        
        
print('Testing on test data set')
loss = 0
num_correct = 0
for im, label in zip(test_images, test_labels):
    _, l, acc = forward(im, label)
    loss += l
    num_correct += acc

    
    
num_tests = len(test_images)
print('Loss on test dataset:', loss / num_tests)
print(' Accuracy on test dataset:', num_correct / num_tests)

t2 = time.time()

print('total time taken is :' , t2 -t1)

#print(conv.t_for_np_loops)
print('time taken for forward prop is',conv.t_for_conv, ' seconds')
print('time taken for forward prop is',conv.t_back_prop, ' seconds')

--- Epoch 1 ---
after epoch1loss is 0.4003378448149679accuracy is0.276
--- Epoch 2 ---
after epoch2loss is 0.1659455947680067accuracy is0.346
--- Epoch 3 ---
after epoch3loss is 0.10378644490645153accuracy is0.372
Testing on test data set
Loss on test dataset: 0.6980520137512619
 Accuracy on test dataset: 0.77
total time taken is : 106.15123319625854
time taken for forward prop is 55.560298681259155  seconds
time taken for forward prop is 39.738919496536255  seconds


# Ignore the blocks below

In [6]:
from random import randint
from PIL import Image
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow, show

def predict_out(image):
    
    out = conv.forward((image / 255) - 0.5)
    out = pool.forward(out)
    out = softmax.forward(out)
    
    return np.argmax(out)



print("test cases: \n\n ")

for j in range(7):
    rand = randint(1000,2000)
    print("\n\ninput image :")
    test_img = np.array(images_ts1[rand]).reshape(28,28)
    imshow(test_img)
    show()
    
    print(" predicted Output" , end = "")
    print(predict_out(test_img))
    
    
    

test cases: 

 


input image :


<Figure size 640x480 with 1 Axes>

 predicted Output2


input image :


<Figure size 640x480 with 1 Axes>

 predicted Output4


input image :


...

 predicted Output4


input image :


<Figure size 640x480 with 1 Axes>

 predicted Output1


input image :


<Figure size 640x480 with 1 Axes>

 predicted Output1


In [7]:
in_array = [1, 2, 3.78, 4, 5.54, 6.6, 7, 8 ] 
print ("Input array : ", in_array) 
  
out_array = np.clip(in_array, a_min = 2, a_max = 6) 
print ("Output array : ", out_array) 


Input array :  [1, 2, 3.78, 4, 5.54, 6.6, 7, 8]
Output array :  [2.   2.   3.78 4.   5.54 6.   6.   6.  ]


In [8]:
import math
f = np.random.randn(8, 3, 3) / 9
print(f)

print('\n')

t3 = time.time()

for i in range(5):
    for i in range(1000):
        
        for i in range(len(f)):
            #print('\n')
            #print(f[i])
            #print('\n')
            for _ in range(4): 

                min = math.inf
                for j in range(len(f[i])):
                    for k in range(len(f[i][j])):
                        if abs(f[i,j,k]) < min:
                            min = abs(f[i,j,k])
                            min_j = j
                            min_k = k

                f[i][min_j][min_k] = math.inf

            for j in range(len(f[i])):
                for k in range(len(f[i][j])):
                    if f[i,j,k] == math.inf:
                        f[i,j,k] = 0



        for i in range(len(f)):

            for j in range(len(f[i])):

                for k in range(len(f[i][j])):

                    if f[i,j,k] > 0 :
                        f[i,j,k] = 1

                    if f[i,j,k] < 0 :
                        f[i,j,k] = -1
                        

for i in range(1000):
        
        for i in range(len(f)):
            #print('\n')
            #print(f[i])
            #print('\n')
            for _ in range(4): 

                min = math.inf
                for j in range(len(f[i])):
                    for k in range(len(f[i][j])):
                        if abs(f[i,j,k]) < min:
                            min = abs(f[i,j,k])
                            min_j = j
                            min_k = k

                f[i][min_j][min_k] = math.inf

            for j in range(len(f[i])):
                for k in range(len(f[i][j])):
                    if f[i,j,k] == math.inf:
                        f[i,j,k] = 0



        for i in range(len(f)):

            for j in range(len(f[i])):

                for k in range(len(f[i][j])):

                    if f[i,j,k] > 0 :
                        f[i,j,k] = 1

                    if f[i,j,k] < 0 :
                        f[i,j,k] = -1
            

t4 = time.time()

print(t4 - t3)
            
    
            

[[[-0.13418057  0.05473408  0.02397852]
  [ 0.22301843 -0.00775578  0.1543018 ]
  [ 0.05514478 -0.11420193 -0.03089557]]

 [[-0.03014475  0.2175895  -0.08035515]
  [-0.03620168 -0.06306285 -0.07655386]
  [ 0.18747339 -0.06825308  0.05832685]]

 [[-0.2430731  -0.04246216 -0.0582466 ]
  [-0.20858788 -0.02920913 -0.11558342]
  [-0.12240005 -0.04662316 -0.02542909]]

 [[-0.0198415  -0.02781205  0.07268217]
  [-0.08001157  0.13476291 -0.03711937]
  [ 0.08289671  0.00061542  0.11181225]]

 [[ 0.12345655  0.03867507  0.10082984]
  [-0.30230827 -0.0590996  -0.10371425]
  [ 0.01629502  0.0156178   0.04448666]]

 [[-0.10074302 -0.04262166  0.1476819 ]
  [-0.10159211  0.03823532 -0.08810532]
  [ 0.05314499  0.045318    0.1243079 ]]

 [[ 0.07112613 -0.01825244 -0.15954592]
  [-0.01216734  0.08590976 -0.04114128]
  [ 0.04783625  0.21813266 -0.042864  ]]

 [[ 0.04651892  0.11231542  0.0482709 ]
  [ 0.04883702 -0.11078608  0.04618817]
  [-0.18954755 -0.04187727  0.05837648]]]


3.3902621269226074


In [9]:
print(conv.t_back_prop)

39.738919496536255


In [10]:
img = train_images[57]/255
a = np.argwhere(img > 0.3)
print(len(a))

94


In [11]:
import numpy as np
p=.5
def ternary_set_conv(weight_matrix):  
    i=0
    w = len(weight_matrix.flatten())
    while(i<int(w*p)):
        b=weight_matrix[weight_matrix!=0] #for remove 0 element and update in b
        n = b.flat[np.abs(b-0).argmin()] #for value nearest to the 0
        indexes=np.where(weight_matrix==n)
        indexes1 = list(zip(indexes[0], indexes[1]))
        print(indexes1)
        weight_matrix[indexes1[0]]=0  
        i=np.sum(weight_matrix==0)
    return weight_matrix

orignal_weight_matrix=np.array([[-0.2,0.3,0.5],
                                [0.9,0.5,0.5],
                                [0.1,-0.6,0.8]])
# print("Origianl metrix: ", orignal_weight_matrix)

print("Ternary Set Conversion: ", ternary_set_conv(orignal_weight_matrix))


[(2, 0)]
[(0, 0)]
[(0, 1)]
[(0, 2), (1, 1), (1, 2)]
Ternary Set Conversion:  [[ 0.   0.   0. ]
 [ 0.9  0.5  0.5]
 [ 0.  -0.6  0.8]]


In [12]:
--- Epoch 1 ---
after epoch1loss is 0.8006655156999186accuracy is0.76
--- Epoch 2 ---
after epoch2loss is 0.37451504937217084accuracy is0.887
--- Epoch 3 ---
after epoch3loss is 0.2749946300165077accuracy is0.93
Testing on test data set
Loss on test dataset: 0.4027378477931141
 Accuracy on test dataset: 0.877
total time taken is : 508.10179257392883
time taken for forward prop is 290.22868490219116  seconds
time taken for forward prop is 171.6409204006195  seconds

SyntaxError: invalid syntax (<ipython-input-12-dbd146599c34>, line 1)